In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# We'll use sentence-transformers for text embeddings.
!pip install -U sentence-transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Load the FAQs Dataset
import pandas as pd
faqs_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/random/Interview_task/Headless/FAQs.csv")
faqs_df

,Question,Answer
0,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.
1,Where was he born?,"He was born in Ulm, Germany."
2,When did he die?,"He died 18 April 1955 in Princeton, New Jersey..."
3,Who were his parents?,His father was Hermann Einstein and his mother...
4,Did he have any sisters and brothers?,He had one sister named Maja.
5,Did he marry and have children?,He was married to Mileva Marić between 1903 an...
6,Where did he receive his education?,He received his main education at the followin...
7,When was Albert Einstein awarded the Nobel Pri...,"The Nobel Prize Awarding Institution, the Roya..."
8,Did Albert Einstein attend the Nobel Prize Awa...,The Nobel Prize was announced on 9 November 19...
9,For what did he receive the Nobel Prize?,Einstein was rewarded for his many contributio...


In [ ]:
# Load the FAQs-Test Dataset
faq_test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/random/Interview_task/Headless/FAQs_test.csv")
faq_test_df

,Question
0,What is the date of his death?
1,Did Einstein have siblings?
2,Who was his wife?
3,What was Einstein's father's name?
4,At what institutions did he study?


In [ ]:
# I've added a couple of pretrained model here.
# You can find recommended sentence embedding models from the dropdown
from IPython.display import *
import ipywidgets as widgets
from ipywidgets import *
model_name = "all-MiniLM-L12-v2"
model_name = Dropdown(options = ['all-MiniLM-L12-v2','all-MiniLM-L6-v2','all-distilroberta-v1','all-mpnet-base-v2','multi-qa-distilbert-cos-v1 '])
model_name

Dropdown(options=('all-MiniLM-L12-v2', 'all-MiniLM-L6-v2', 'all-distilroberta-v1', 'all-mpnet-base-v2', 'multi…

In [ ]:
# Load the model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(model_name.value)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
# Make an array of all the Questions
faqs = faqs_df['Question'].unique()
faqs

array(['When was Albert Einstein born?', 'Where was he born?',
       'When did he die?', 'Who were his parents?',
       'Did he have any sisters and brothers?',
       'Did he marry and have children?',
       'Where did he receive his education?',
       'When was Albert Einstein awarded the Nobel Prize in Physics?',
       'Did Albert Einstein attend the Nobel Prize Award Ceremony?',
       'For what did he receive the Nobel Prize?'], dtype=object)

In [ ]:
# Make an array of all the Test Questions
faqs_test = faq_test_df['Question'].unique()
faqs_test

array(['What is the date of his death?', 'Did Einstein have siblings?',
       'Who was his wife?', "What was Einstein's father's name?",
       'At what institutions did he study?'], dtype=object)

In [ ]:
# Sentences encode.
faqs_vect = model.encode(faqs)
faqs_test_vect = model.encode(faqs_test)

In [ ]:
# Check the vector shape
print(faqs_vect.shape)
print(faqs_test_vect.shape)

(10, 384)
(5, 384)


In [ ]:
faqs_test_vect

array([[ 0.01096014,  0.08328179, -0.06373304, ...,  0.02939607,
         0.02383848, -0.01802064],
       [-0.00581924,  0.01827138, -0.05965833, ..., -0.04684462,
         0.07210156, -0.05503883],
       [-0.06763846,  0.07056388, -0.04037826, ...,  0.00411596,
        -0.01609732, -0.03327011],
       [-0.02582397,  0.00313276, -0.04120911, ..., -0.01027508,
         0.08042558, -0.01450535],
       [ 0.0989913 ,  0.05692837, -0.05295782, ..., -0.05165843,
        -0.01619162, -0.03793705]], dtype=float32)

In [ ]:
# Check the cosine similarity of two sentence
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([faqs_test_vect[2]], [faqs_vect[5]])

array([[0.63131356]], dtype=float32)

In [ ]:
print(faqs_test[2])

Who was his wife?


In [ ]:
print(faqs[5])

Did he marry and have children?


In [ ]:
import csv

# We'll travers all the Question for each test question and check the similarity score
rows = []
for i in range(len(faqs_test)): 
    print("Test Ques: ", faqs_test[i])
    row = []
    highest_similarity_score = 0 # initial similarity score
    best_match_faq = "" # initial best mached question 
    for j in range(len(faqs)): 
        similarity_score = cosine_similarity([faqs_test_vect[i]], [faqs_vect[j]])
        if similarity_score > highest_similarity_score:  
            highest_similarity_score = similarity_score[0][0] # update the highest_similarity_score
            best_match_faq = faqs[j] # update the best mached question 

    row.extend([faqs_test[i], best_match_faq, highest_similarity_score, faqs_df.loc[faqs_df['Question'] == best_match_faq, 'Answer'].squeeze()])
    rows.append(row)
    print("Best matched question: ", best_match_faq)
    print("Similarity Score: ", highest_similarity_score)
    print("Answer: ", faqs_df.loc[faqs_df['Question'] == best_match_faq, 'Answer'].squeeze()) # Fetch the answer of best matched question from FAQs dataset 
    print()
# name of csv file 
filename = "result.csv"
fields = ['Test Question', 'Main Question', 'Similarity Score', 'Answer'] 

    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)

from google.colab import files
files.download('result.csv')



Test Ques:  What is the date of his death?
Best matched question:  When did he die?
Similarity Score:  0.8527833
Answer:  He died 18 April 1955 in Princeton, New Jersey, USA.

Test Ques:  Did Einstein have siblings?
Best matched question:  When was Albert Einstein born?
Similarity Score:  0.72801024
Answer:  Albert Einstein was born on 14 March 1879.

Test Ques:  Who was his wife?
Best matched question:  Did he marry and have children?
Similarity Score:  0.63131356
Answer:  He was married to Mileva Marić between 1903 and 1919. They had three children, Lieserl (born 1902), Hans Albert (born 1904) and Eduard (born 1910). He married Elsa Löwenthal in 1919 and they lived together until her death in 1936.

Test Ques:  What was Einstein's father's name?
Best matched question:  When was Albert Einstein born?
Similarity Score:  0.75009066
Answer:  Albert Einstein was born on 14 March 1879.

Test Ques:  At what institutions did he study?
Best matched question:  Where did he receive his educatio

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>